# Use semantic chunking in Document Intelligence with Azure AI Search

This sample demonstrates how to use the [semantic chunking capabilities of Document Intelligence](https://learn.microsoft.com/azure/ai-services/document-intelligence/concept-retrieval-augmented-generation) with Azure AI Search.

Semantic chunking divides the text into chunks based on semantic understanding. Division boundaries are focused on sentence subject, maintaining semantic consistency within each chunk. It's useful for text summarization, sentiment analysis, and document classification tasks. It can significantly improve the experience of conversational chat and generative AI because the upstream inputs available to the model are of higher quality.

The sample uses a custom skill to add semantic chunking to an indexing pipeline in Azure AI Search. The sample content might be familiar to you if you've run other Azure AI Search samples in the past. It's a collection of PDFs about employee healthcare, benefits, and employment guidance. In constrast with other samples that use fixed-size data chunking, this sample uses semantic chunking during indexing.

This notebook takes you directly to queries that execute on the search engine so that you can evaluate the quality of the response based on the value added by semantic chunking. There's no generative AI or chat step to obscure the effect of semantic chunking. 

If you want to learn more about how the solution is built, review the source code in this folder. To try out semantic chunking with chat models and generative AI, you can adapt the [RAG tutorial](https://github.com/Azure-Samples/azure-search-python-samples/tree/main/Tutorial-RAG), replacing the fixed-length data chunking with the semantic chunking component of this sample.

## Prerequisites

+ Follow the instructions in the [readme](./readme.md) to deploy all Azure resources, including sample data and the search index.

+ Check your search service to make sure the *document-intelligence-index* exists. If you don't see an index, revisit the readme and run the `setup_search_service` script.

+ Don't add an `.env` file to this folder. Environment variables are read from the `azd` deployment. You can type `azd env get-values` to review the variables.

## Grant permissions

The deployment script creates an Azure AI Search resource that's configured for Azure role-based access. By default, you inherit the **Owner** role assignment of your Azure subscription. This section explains which additional permissions are required to run the queries.

### Assign roles on Azure AI Search

If you are an **Owner**, you have permission to create and manage objects on your search service, but you don't have read-access to indexes.

You can use the Azure portal and the **Access Control (IAM)** page to give yourself **Search Index Data Reader** permissions. Or, [use the Azure CLI to assign roles](https://learn.microsoft.com/azure/role-based-access-control/role-assignments-cli) for a command line approach. Currently, you can't use `azd` to assign roles.

1. Modify the following command to use valid values:

   ```azurecli
    az role assignment create --assignee "55555555-5555-5555-5555-555555555555" --role "Search Index Data Reader" --scope "/subscriptions/00000000-0000-0000-0000-000000000000/resourceGroups/rg-YOUR-ENVIRONMENT-NAME/providers/Microsoft.Search/searchServices/srch-RANDOM-STRING"
   ```

   - The assignee is your personal identity token or your account name such as such as *patlong@contoso.com*.
   - The subscription ID can be obtained from `az account show`.
   - If you used the script to deploy resources, check the Azure portal or the status message of the deployment script for the resource group and search service name.

1. Paste the modified into the command line and press **Enter**.

If successful, the command returns information about the role assignment.

Later, if you run queries and get an "HttpResponseError: Operation returned an invalid status 'Forbidden'" error, revisit this step to make sure you completed this step.

### Assign roles on Azure OpenAI

The queries in this notebook are provided in plain text ("What's a performance review?"). At query time, the search engine calls a vectorizer to generate an embedding of the query string, which is then passed back to the search engine for vector search over the vector fields in the index.

To give the search service access to the embedding model, assign to it the **Cognitive Services OpenAI User** permissions on Azure OpenAI.

1. Modify the following command to use valid values:

   ```azurecli
   az role assignment create --assignee "55555555-5555-5555-5555-555555555555" --role "Cognitive Services OpenAI User" --scope "/subscriptions/00000000-0000-0000-0000-000000000000/resourceGroups/rg-YOUR-ENVIRONMENT-NAME/providers/Microsoft.CognitiveServices/accounts/cog-RANDOM-STRING"
   ```

   - The assignee is the system-assigned managed identity of your Azure AI Search service.
   - The subscription ID can be obtained from `az account show`.
   - If you used the script to deploy resources, check the Azure portal or the status message of the deployment script for the resource group and Azure OpenAI account name.

1. Paste the modified into the command line and press **Enter**.

Later, if you run queries and get "HttpResponseError: () Could not complete vectorization action. The vectorization endpoint returned status code '401' (Unauthorized)", followed by "Message: Could not complete vectorization action. The vectorization endpoint returned status code '401' (Unauthorized).", then revisit this step to give yourself permissions.

## Get started

Install the packages and load the libraries that are necessary for running the queries in this notebook.

In [ ]:
! pip install azure-search-documents==11.6.0b4 --quiet
! pip install python-dotenv azure-identity --quiet

In [ ]:
# Load all environment variables from the azd deployment
import subprocess
from io import StringIO
from dotenv import load_dotenv
result = subprocess.run(["azd", "env", "get-values"], stdout=subprocess.PIPE)
load_dotenv(stream=StringIO(result.stdout.decode("utf-8")))

In [22]:
import os
search_url = f"https://{os.environ['AZURE_SEARCH_SERVICE']}.search.windows.net"

## Perform a vector similarity search

This example shows a pure vector search using the vectorizable text query, all you need to do is pass in text and your vectorizer will handle the query vectorization.

In [25]:
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizableTextQuery
from azure.identity import DefaultAzureCredential
# Pure Vector Search
query = "What's a performance review?"  
  
search_client = SearchClient(search_url, "document-intelligence-index", credential=DefaultAzureCredential())
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="vector", exhaustive=True)
# Use the below query to pass in the raw vector query instead of the query vectorization
# vector_query = RawVectorQuery(vector=generate_embeddings(query), k_nearest_neighbors=3, fields="vector")
  
results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["parent_id", "chunk_id", "chunk_headers", "chunk"],
    top=1
)  
  
for result in results:
    print(f"parent_id: {result['parent_id']}")  
    print(f"Score: {result['@search.score']}") 
    print(f"Chunk Headers: {result['chunk_headers']}")
    print(f"Content: {result['chunk']}")  


## Perform a hybrid search

Search using text and vectors combined for more relevant results

In [19]:
# Hybrid Search
query = "What's the difference between the health plans?"  
  
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="vector", exhaustive=True)
  
results = search_client.search(  
    search_text=query,  
    vector_queries= [vector_query],
    select=["parent_id", "chunk_id", "chunk_headers", "chunk"],
    top=1
)  
  
for result in results:  
    print(f"parent_id: {result['parent_id']}")  
    print(f"chunk_id: {result['chunk_id']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Chunk Headers: {result['chunk_headers']}")
    print(f"Content: {result['chunk']}")  


## Use chunk headers to improve search results

Semantic chunking retrieves section headers if they are available. Use them to improve your search results

Note that semantic chunking from document intelligence automatically converts tables to Markdown form

In [16]:
# Hybrid Search
query = "How does the cost between health plans compare?"  
  
vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="vector", exhaustive=True)
  
results = search_client.search(  
    search_text=query,  
    vector_queries= [vector_query],
    select=["parent_id", "chunk_id", "chunk_headers", "chunk"],
    search_fields=["chunk", "chunk_headers"],
    top=1
)  
  
for result in results:  
    print(f"parent_id: {result['parent_id']}")  
    print(f"chunk_id: {result['chunk_id']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Chunk Headers: {result['chunk_headers']}")
    print(f"Content: {result['chunk']}")  


## Perform a hybrid search + Semantic reranking

In [ ]:
from azure.search.documents.models import QueryType, QueryCaptionType, QueryAnswerType

# Semantic Hybrid Search
query = "What's a performance review?"

vector_query = VectorizableTextQuery(text=query, k_nearest_neighbors=50, fields="vector", exhaustive=True)

results = search_client.search(  
    search_text=query,
    vector_queries=[vector_query],
    select=["parent_id", "chunk_id", "chunk"],
    query_type=QueryType.SEMANTIC,  semantic_configuration_name='my-semantic-config', query_caption=QueryCaptionType.EXTRACTIVE, query_answer=QueryAnswerType.EXTRACTIVE,
    top=2
)

semantic_answers = results.get_answers()
for answer in semantic_answers:
    if answer.highlights:
        print(f"Semantic Answer: {answer.highlights}")
    else:
        print(f"Semantic Answer: {answer.text}")
    print(f"Semantic Answer Score: {answer.score}\n")

for result in results:
    print(f"parent_id: {result['parent_id']}")  
    print(f"chunk_id: {result['chunk_id']}")  
    print(f"Score: {result['@search.score']}")  
    print(f"Content: {result['chunk']}")  

    captions = result["@search.captions"]
    if captions:
        caption = captions[0]
        if caption.highlights:
            print(f"Caption: {caption.highlights}\n")
        else:
            print(f"Caption: {caption.text}\n")
